In [13]:
import io
import numpy as np
import pandas as pd
import math

In [14]:
dados = pd.read_csv("/content/dataset.csv")

In [15]:
def dbm2mw(v):
  return 10 ** (v/10)
def mw2dbm(v):
  return 10 * np.log10(v)

def potenciaReal(pw):
  if pw == 20: return 15;
  elif pw == 17: return 14;
  elif pw == 14: return 12;
  elif pw == 11: return 11;

In [16]:
def fatorCarrecao(row):
  if (row['sf'] == 10 and row['pw'] == 20): return 15.85
  elif (row['sf'] == 9 and row['pw'] == 20): return 15.19
  elif (row['sf'] == 8 and row['pw'] == 20): return 14.73
  elif (row['sf'] == 7 and row['pw'] == 20): return 9.86

  elif (row['sf'] == 10 and row['pw'] == 17): return 14.22
  elif (row['sf'] == 9 and row['pw'] == 17): return 13.59
  elif (row['sf'] == 8 and row['pw'] == 17): return 13.04
  elif (row['sf'] == 7 and row['pw'] == 17): return 8.08

  elif (row['sf'] == 10 and row['pw'] == 14): return 15.36
  elif (row['sf'] == 9 and row['pw'] == 14): return 15.32
  elif (row['sf'] == 8 and row['pw'] == 14): return 14.77
  elif (row['sf'] == 7 and row['pw'] == 14): return 9.00

  elif (row['sf'] == 10 and row['pw'] == 11): return 12.19
  elif (row['sf'] == 9 and row['pw'] == 11): return 11.87
  elif (row['sf'] == 8 and row['pw'] == 11): return 11.02
  elif (row['sf'] == 7 and row['pw'] == 11): return 7.26

In [17]:
def fspl(row):
  return round(20 * math.log10(row['distancia']) + 20 * math.log10(row['freq'] * 10 ** 6) + 20 * math.log10(4 * math.pi / 299792458), 2)

In [18]:
posicoes = dados.groupby(['id','data','origem_antena','origem_altura','destino_antena','destino_altura','origem_lat','origem_lng','destino_lat','destino_lng','distancia','freq','pw','sf','cr','cobertura'])
mean = posicoes['rssi'].mean().round(2).reset_index(name='RSSI')

mean['potenciaReal'] = mean.apply(lambda row: potenciaReal(row['pw']), axis=1)
mean['SNR'] = posicoes['snr'].mean().round(2).reset_index(name='SNR')['SNR']
mean['EIRP'] = mean['potenciaReal'] + mean['origem_antena']
mean['IRL'] = mean['RSSI'] + mean['destino_antena']
mean['PL'] = mean['EIRP'] - mean['IRL']
mean['FSPL'] = mean.apply(lambda row: fspl(row), axis=1)
mean['FC'] = mean.apply(lambda row: fatorCarrecao(row), axis=1)
mean['PL-FC'] = mean['PL'] - mean['FC']

mean

,id,data,origem_antena,origem_altura,destino_antena,destino_altura,origem_lat,origem_lng,destino_lat,destino_lng,...,cobertura,RSSI,potenciaReal,SNR,EIRP,IRL,PL,FSPL,FC,PL-FC
0,3183,2024-04-25,3,1.5,3,1.5,-31.763630,-52.257740,-31.765390,-52.256850,...,Mista,-90.10,15,6.70,18,-87.10,105.10,78.12,15.85,89.25
1,3184,2023-04-25,3,1.5,3,1.5,-31.764030,-52.257570,-31.765390,-52.256850,...,Mista,-75.44,15,7.00,18,-72.44,90.44,75.96,15.85,74.59
2,3201,2023-05-22,3,1.5,3,1.5,-32.217871,-52.188389,-32.217881,-52.188405,...,Praia,-26.85,15,6.54,18,-23.85,41.85,37.58,15.85,26.00
3,3202,2023-05-22,3,1.5,3,1.5,-32.217871,-52.188389,-32.217881,-52.188405,...,Praia,-26.40,15,7.62,18,-23.40,41.40,37.58,15.19,26.21
4,3203,2023-05-22,3,1.5,3,1.5,-32.217871,-52.188389,-32.217881,-52.188405,...,Praia,-25.64,15,7.35,18,-22.64,40.64,37.58,14.73,25.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,3484,2023-06-02,3,1.5,3,1.5,-31.763804,-52.257577,-31.766350,-52.256131,...,Mista,-75.50,12,6.30,15,-72.50,87.50,81.50,9.00,78.50
286,3485,2023-06-02,3,1.5,3,1.5,-31.763804,-52.257577,-31.766350,-52.256131,...,Mista,-79.70,11,8.00,14,-76.70,90.70,81.50,12.19,78.51
287,3486,2023-06-02,3,1.5,3,1.5,-31.763804,-52.257577,-31.766350,-52.256131,...,Mista,-79.80,11,8.20,14,-76.80,90.80,81.50,11.87,78.93
288,3487,2023-06-02,3,1.5,3,1.5,-31.763804,-52.257577,-31.766350,-52.256131,...,Mista,-80.40,11,7.40,14,-77.40,91.40,81.50,11.02,80.38


In [19]:
print('Total de registros:', dados['cobertura'].count())
print(dados.groupby(['cobertura'])['cobertura'].count())

Total de registros: 9776
cobertura
Floresta     530
Mista        364
Praia       7884
Rua          998
Name: cobertura, dtype: int64


In [20]:
from scipy.optimize import curve_fit

def objctiveFunctionLD(d, A, n, d0):
    if d0 <= 0: d0 = 0.000001
    return A + 10 * n * np.log10(d/d0)

valores_n = pd.DataFrame({'cobertura': [], 'pw': [], 'sf': [], 'PLd0':[], 'd0':[], 'n':[]})
valores_n = valores_n.astype({'cobertura': str,'pw': int,'sf':int, 'PLd0':float,'d0':float,'n':float})

coberturas = ['Praia', 'Floresta', 'Rua']
pws = [20,17,14,11]
sfs = [10,9,8,7]
for cobertura in coberturas:
  for pw in pws:
    for sf in sfs:
      selecionados = (mean['pw'] == pw) & (mean['sf'] == sf) & (mean['cobertura'] == cobertura)

      params, _ = curve_fit(objctiveFunctionLD, mean[selecionados]['distancia'], mean[selecionados]['PL-FC'], p0=[31.56, 2, 1])
      PLd0, n, d0 = params

      novo_n = pd.DataFrame({
          'cobertura': cobertura,
          'pw': pw,
          'sf': sf,
          'PLd0': round(PLd0, 2),
          'd0': round(d0, 2),
          'n': round(n, 2)
      }, index=[0])

      valores_n = pd.concat([valores_n, novo_n], ignore_index=True)

print(valores_n)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estima

   cobertura  pw  sf   PLd0    d0     n
0      Praia  20  10  25.11  2.16  2.35
1      Praia  20   9  26.31  2.35  2.33
2      Praia  20   8  24.86  2.12  2.35
3      Praia  20   7  23.20  1.63  2.39
4      Praia  17  10  23.52  1.42  2.32
5      Praia  17   9  32.40  3.42  2.33
6      Praia  17   8  24.27  1.57  2.33
7      Praia  17   7  21.01  1.10  2.45
8      Praia  14  10  26.78  2.05  2.30
9      Praia  14   9  23.60  1.56  2.30
10     Praia  14   8  27.39  2.33  2.32
11     Praia  14   7  29.46  2.47  2.43
12     Praia  11  10  24.58  1.02  2.31
13     Praia  11   9  28.98  1.61  2.31
14     Praia  11   8  29.88  1.72  2.30
15     Praia  11   7  24.78  1.15  2.44
16  Floresta  20  10   8.22  4.93  4.31
17  Floresta  20   9   0.47  3.70  4.49
18  Floresta  20   8  -0.11  3.60  4.46
19  Floresta  20   7   3.35  4.47  4.67
20  Floresta  17  10   8.56  4.31  4.26
21  Floresta  17   9   0.05  2.73  4.28
22  Floresta  17   8   2.67  3.13  4.26
23  Floresta  17   7   5.36  4.60  4.78


In [21]:
def log_distance(row, valores_n):
  if row['cobertura'] == 'Mista': return;

  selecionados = (valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == row['cobertura'])

  n = valores_n[selecionados]['n']
  d0 = valores_n[selecionados]['d0']
  PLd0 = valores_n[selecionados]['PLd0']

  return (PLd0 + 10 * n * np.log10(row['distancia']/d0))

mean['Log-Distance'] = mean.apply(lambda row: log_distance(row, valores_n), axis=1)

In [22]:
def sateLoc(row, valores_n):
  if row['cobertura'] == 'Mista': return;

  selecionados = (valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == row['cobertura'])

  n = valores_n[selecionados]['n']
  d0 = valores_n[selecionados]['d0']
  PLd0 = valores_n[selecionados]['PLd0']

  return (PLd0 + 10 * n * np.log10(row['distancia']/d0))

mean['sateLoc'] = mean.apply(lambda row: sateLoc(row, valores_n), axis=1)

In [23]:
def maxCover(row, valores_n):
  if row['cobertura'] == 'Mista': return;

  selecionados = (valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == row['cobertura'])
  n = valores_n[selecionados]['n']

  wavelength = 299792458 / row['freq']

  K = dbm2mw(row['origem_antena']) * dbm2mw(row['destino_antena']) * ((wavelength / (4 * math.pi)) ** 2)

  Pr = mw2dbm(K * dbm2mw(row['potenciaReal']) * (1 / (row['distancia'] ** n)))

  IRL = Pr + row['destino_antena']
  EIRP = row['potenciaReal'] + row['origem_antena']
  return IRL - EIRP

mean['maxCover'] = mean.apply(lambda row: maxCover(row, valores_n), axis=1)

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

desempenho_modelos = pd.DataFrame({'modelo': [], 'r2': [], 'MAE': [], 'MAPE':[], 'MSE':[], 'RMSE':[]})

desempenho_modelos = pd.concat([desempenho_modelos,  pd.DataFrame({
    'modelo': 'FSPL',
    'r2': round(r2_score(mean['PL-FC'], mean['FSPL']), 2),
    'MAE': round(mean_absolute_error(mean['PL-FC'], mean['FSPL']), 2),
    'MAPE': round(mean_absolute_percentage_error(mean['PL-FC'], mean['FSPL']), 2),
    'MSE': round(mean_squared_error(mean['PL-FC'], mean['FSPL']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(mean['PL-FC'], mean['FSPL'])), 2)
}, index=[0])], ignore_index=True)

enlaces_cobertura_unica = mean[(mean['cobertura'] != 'Mista')]

desempenho_modelos = pd.concat([desempenho_modelos, pd.DataFrame({
    'modelo': 'Log-Distance',
    'r2': round(r2_score(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos = pd.concat([desempenho_modelos, pd.DataFrame({
    'modelo': 'SateLoc',
    'r2': round(r2_score(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['sateLoc']), 2),
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['sateLoc']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['sateLoc']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['sateLoc']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['sateLoc'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos = pd.concat([desempenho_modelos, pd.DataFrame({
    'modelo': 'Max-Cover',
    'r2': round(r2_score(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['maxCover']), 2),
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['maxCover']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['maxCover']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['maxCover']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['maxCover'])), 2)
}, index=[0])], ignore_index=True)

print(desempenho_modelos)

         modelo    r2    MAE  MAPE      MSE   RMSE
0          FSPL  0.83   6.98  0.14    63.61   7.98
1  Log-Distance  0.98   2.00  0.04     6.69   2.59
2       SateLoc  0.98   2.00  0.04     6.69   2.59
3     Max-Cover -5.69  44.48  0.83  2511.86  50.12


In [29]:
enlaces_cobertura_mista = pd.DataFrame({
    'PL-FC' : [84.55,87.44,85.58,87.52,82.74,87.10,86.71,89.29,86.55,87.94,84.28,87.12,82.54,86.50,84.51,89.54],
    'FSPL'  : [76.90,76.90,76.90,76.90,76.90,76.90,76.90,76.90,81.50,81.50,81.50,81.50,81.50,81.50,81.50,81.50],
    'sateLoc' : [78.69 ,85.23 ,77.96 ,86.48 ,77.46 ,83.58 ,79.69 ,84.71 ,73.13 ,79.64 ,72.43 ,80.75 ,71.70 ,76.90 ,74.27 ,78.81]
})

desempenho_modelos_CM = pd.DataFrame({'modelo': [], 'r2': [], 'MAE': [], 'MAPE':[], 'MSE':[], 'RMSE':[]})

desempenho_modelos_CM = pd.concat([desempenho_modelos_CM,  pd.DataFrame({
    'modelo': 'FSPL',
    'r2': round(r2_score(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['FSPL']), 2),
    'MAE': round(mean_absolute_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['FSPL']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['FSPL']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['FSPL']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['FSPL'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos_CM

,modelo,r2,MAE,MAPE,MSE,RMSE
0,FSPL,-13.84,7.04,0.08,59.48,7.71
